In [5]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("robotics.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'robotics.txt'}, page_content='As more and more robots are designed for specific tasks, this method of classification becomes more relevant. For example, many robots are designed for assembly work, which may not be readily adaptable for other applications. They are termed "assembly robots". For seam welding, some suppliers provide complete welding systems with the robot i.e. the welding equipment along with other material handling facilities like turntables, etc. as an integrated unit. Such an integrated robotic system is called a "welding robot" even though its discrete manipulator unit could be adapted to a variety of tasks. Some robots are specifically designed for heavy load manipulation, and are labeled as "heavy-duty robots".[4]\n\nCurrent and potential applications include:\n\nManufacturing. Robots have been increasingly used in manufacturing since the 1960s. According to the Robotic Industries Association US data, in 2016 the automotive industry wa

In [36]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="simplerag"


In [11]:
from langchain_community.document_loaders import WebBaseLoader

import bs4

web_loader = WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Robotics",),
                           bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                               class_=("mw-body-content")
                           )))
web_documents = web_loader.load()
web_documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Robotics'}, page_content='Design, construction, use, and application of robots\nThis article may relate to a different subject or has undue weight on an aspect of the subject. Specifically, the article goes too much detail on specific types of robot and includes product placement. Please help relocate relevant information and remove irrelevant content. (August 2024)\n\n\nRoboticists with three Mars rover robots. Front and center is the flight spare for the first Mars rover, Sojourner, which landed on Mars in 1997 as part of the Mars Pathfinder Project. On the left is a Mars Exploration Rover (MER) test vehicle that is a working sibling to Spirit and Opportunity, which landed on Mars in 2004. On the right is a test rover for the Mars Science Laboratory, which landed Curiosity on Mars in 2012.\n\nRobotics is the interdisciplinary study and practice of the design, construction, operation, and use of robots.[1]\nWithin mechanical

In [13]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("robotics.pdf")
pdf_documents = pdf_loader.load()
pdf_documents

[Document(metadata={'source': 'robotics.pdf', 'page': 0}, page_content='ENGINEERING ROBOTICS -\nPROFESSIONAL ELECTIVE'),
 Document(metadata={'source': 'robotics.pdf', 'page': 1}, page_content='The field ofrobotics has itsorigins inscience fiction .The term robot was derived\nfrom theEnglish translation ofafantasy play written inCzechoslovakia around 1920 .\nIttook another 40years before themodern technology ofindustrial robotics began .\nToday, Robots are highly automated mechanical manipulators controlled by\ncomputers .\nRobotics :-\nRobotics isanapplied engineering science that has been referred toasa\ncombination ofmachine tool technology and computer science .Itincludes machine\ndesign, production theory, micro electronics, computer programming &artificial\nintelligence .\nOR\n"Robotics" isdefined asthe science ofdesigning and building Robots which are\nsuitable for real life application inautomated manufacturing and other non-\nmanufacturing environments .INTRODUCTION'),
 Documen

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = web_documents + pdf_documents + text_documents
documents = text_splitter.split_documents(docs)
documents[:3]

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Robotics'}, page_content='Design, construction, use, and application of robots\nThis article may relate to a different subject or has undue weight on an aspect of the subject. Specifically, the article goes too much detail on specific types of robot and includes product placement. Please help relocate relevant information and remove irrelevant content. (August 2024)\n\n\nRoboticists with three Mars rover robots. Front and center is the flight spare for the first Mars rover, Sojourner, which landed on Mars in 1997 as part of the Mars Pathfinder Project. On the left is a Mars Exploration Rover (MER) test vehicle that is a working sibling to Spirit and Opportunity, which landed on Mars in 2004. On the right is a test rover for the Mars Science Laboratory, which landed Curiosity on Mars in 2012.'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Robotics'}, page_content='Robotics is the interdisciplinary study and pr

c:\Users\HP\Downloads\krish3\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Hugging Face model for embeddings
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy().tolist()  


class CustomEmbeddings:
    def embed_documents(self, texts):
        return [get_embedding(chunk.page_content) for chunk in documents]
    def embed_query(self, text):
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        # Convert the embedding to a list
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy().tolist()


embedding_model = CustomEmbeddings()


In [55]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(documents, embedding_model)

In [56]:
query = "Robots have been increasingly used in manufacturing since the 1960s. According to"
result = db.similarity_search(query)
result

[Document(metadata={'source': 'robotics.txt'}, page_content='Manufacturing. Robots have been increasingly used in manufacturing since the 1960s. According to the Robotic Industries Association US data, in 2016 the automotive industry was the main customer of industrial robots with 52% of total sales.[5] In the auto industry, they can amount for more than half of the "labor". There are even "lights off" factories such as an IBM keyboard manufacturing factory in Texas that was fully automated as early as 2003.[6]\nAutonomous transport including airplane autopilot and self-driving cars\nDomestic robots including robotic vacuum cleaners, robotic lawn mowers, dishwasher loading[7] and flatbread baking.[8]\nConstruction robots. Construction robots can be separated into three types: traditional robots, robotic arm, and robotic exoskeleton.[9]\nAutomated mining.\nSpace exploration, including Mars rovers.\nEnergy applications including cleanup of nuclear contaminated areas[a]; and cleaning sola

In [59]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context.
                            Think step by step before providing a detailed anser.
                            i will tip you if the user finds the answer helpful.
                            <context>
                            {context}
                            </context>
                            Question: {input}
            
""")

In [60]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
document_chain = create_stuff_documents_chain(ChatGroq(), prompt)

In [62]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'CustomEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000024F9751B7D0>, search_kwargs={})

In [65]:
from langchain.chains.retrieval import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [70]:
response = retrieval_chain.invoke({"input": query})
response

{'input': 'Robots have been increasingly used in manufacturing since the 1960s. According to',
 'context': [Document(metadata={'source': 'robotics.txt'}, page_content='Manufacturing. Robots have been increasingly used in manufacturing since the 1960s. According to the Robotic Industries Association US data, in 2016 the automotive industry was the main customer of industrial robots with 52% of total sales.[5] In the auto industry, they can amount for more than half of the "labor". There are even "lights off" factories such as an IBM keyboard manufacturing factory in Texas that was fully automated as early as 2003.[6]\nAutonomous transport including airplane autopilot and self-driving cars\nDomestic robots including robotic vacuum cleaners, robotic lawn mowers, dishwasher loading[7] and flatbread baking.[8]\nConstruction robots. Construction robots can be separated into three types: traditional robots, robotic arm, and robotic exoskeleton.[9]\nAutomated mining.\nSpace exploration, includ

In [69]:
response['answer']

'the provided context, robots have seen significant adoption in the manufacturing sector since the 1960s. This trend is supported by data from the Robotic Industries Association, which indicates that in 2016, the automotive industry was the leading customer of industrial robots, accounting for 52% of total sales. Robots are so prevalent in the auto industry that they often make up more than half of the labor force. In fact, there are instances of fully automated "lights off" factories, such as an IBM keyboard manufacturing facility in Texas, which became operational as early as 2003.\n\nThe context also mentions various other sectors where robots are being utilized, such as:\n\n1. Autonomous transport, including airplane autopilot and self-driving cars\n2. Domestic robots, such as robotic vacuum cleaners, robotic lawn mowers, and automated dishwasher loading\n3. Construction robots, which can be categorized into traditional robots, robotic arms, and robotic exoskeletons\n4. Automated m